### Imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Load the beta-VAE module
import bvae_model as bvae
import spectrum_preprocessing as sp

### Load raw spectra

In [2]:
#your path

path = "C:/Users/Zach/Documents/GitHub/betaVAE_Training/spectral_data/SMP65#013 21d 920um.csv"

In [3]:
filename = path.split("/")[-1]
filename

'SMP65#013 21d 920um.csv'

In [4]:
df = pd.read_csv(path)
print(f"Dataframe shape: {df.shape}")

Dataframe shape: (49152, 509)


In [5]:
df

,map_x,map_y,Sample Name,1981.7 - 2095.8,1799.3865662863864,1797.8025992385988,1796.2186321908116,1794.6346651430244,1793.0506980952368,1791.4667310474497,...,915.5329536210661,913.948986573279,912.3650195254918,910.7810524777042,909.197085429917,907.6131183821294,906.0291513343423,904.4451842865551,902.8612172387675,901.2772501909803
0,39338.3,23186.4,21d 920um,0.027348,0,0.004199,0.003836,0.001615,0.000000,0.000460,...,0.030025,0.044883,0.048524,0.040603,0.026140,0.012403,0.006344,0.009327,0.011530,0
1,39343.7,23186.4,21d 920um,-0.061878,0,0.002311,0.008880,0.019817,0.019869,0.000000,...,0.345553,0.204586,0.034959,0.000000,0.058349,0.154615,0.159013,0.101417,0.056635,0
2,39349.2,23186.4,21d 920um,0.028535,0,0.000000,0.029319,0.071751,0.093872,0.086842,...,0.278553,0.240941,0.222558,0.208304,0.176580,0.134690,0.100347,0.069245,0.031951,0
3,39354.6,23186.4,21d 920um,-0.096200,0,0.011647,0.019450,0.024374,0.026269,0.021104,...,0.013188,0.007545,0.013215,0.017943,0.014202,0.010472,0.012209,0.012503,0.006304,0
4,39360.0,23186.4,21d 920um,-0.020703,0,0.000000,0.000000,0.002668,0.006734,0.010138,...,0.027575,0.039356,0.047574,0.048180,0.038509,0.019920,0.002509,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,41397.6,23875.0,21d 920um,0.011488,0,0.000000,0.016564,0.021799,0.029532,0.063732,...,0.174029,0.074823,0.000000,0.000000,0.037740,0.054078,0.035145,0.025458,0.025206,0
49148,41403.0,23875.0,21d 920um,0.034984,0,0.023373,0.039505,0.047652,0.080177,0.155153,...,0.041400,0.050056,0.045774,0.024974,0.000000,0.006214,0.049741,0.074907,0.044891,0
49149,41408.5,23875.0,21d 920um,0.128355,0,0.029334,0.057110,0.078329,0.086692,0.067475,...,0.096262,0.093465,0.073657,0.040715,0.011877,0.000000,0.000000,0.000000,0.003876,0
49150,41413.9,23875.0,21d 920um,0.154755,0,0.000000,0.000000,0.000272,0.007937,0.015552,...,0.024896,0.033378,0.040071,0.038120,0.028717,0.016131,0.004970,0.000000,0.000000,0


# Sort the wavenumber columns and data and put in a dataframe
- necessary for input to pipeline below

In [6]:
wavenumber = df.columns.values[4:].astype(float)
wavenumber.sort()
wavenumber = wavenumber.astype(str)
spectra_df = df[wavenumber].copy()
np.dtype(spectra_df.columns.values.astype(float)[0])

dtype('float64')

### Run the pipeline
The `bvae.pipeline()` returns the interpolated wavenumber array; the interpolated absorbance array; the beta-model encodings L1 L2 and L3; the reconstruction MSE; and the reconstructed spectrum.

In [7]:
L1_list=[]
L2_list=[]
L3_list=[]
MSE_list=[]
wavenumber_list=[]
absorbance_list=[]
reconstructed_list=[]
for i in range(spectra_df.shape[0]):    
    f = spectra_df.columns.values[:].astype('float32')
    a = spectra_df.iloc[i,:].values
    try:
        wavenumber, absorbance, encodings, mse, reconstructed = bvae.pipeline(f, a)
        L1 = encodings[0]
        L2 = encodings[1]
        L3 = encodings[2]
    except Exception as e:
        print(e)
        L1 = 0
        L2 = 0
        L3 = 0
        mse = 100
        reconstructed = np.zeros(len(wavenumber))
        
    L1_list.append(L1)
    L2_list.append(L2)
    L3_list.append(L3)
    MSE_list.append(mse)
    absorbance_list.append(absorbance)
    reconstructed_list.append(reconstructed)
    
    #print(encodings)
    #print(mse)
    #plt.plot(absorbance,c='b')
    #plt.plot(reconstructed,c='r')
    #plt.show()

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * [<tf.Tensor 'inputs:0' shape=(1063,) dtype=float32>]
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 802), dtype=tf.float32, name='spectra_input')
    * True
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * TensorSpec(shape=(None, 802), dtype=tf.float32, name='spectra_input')
    * False
    * None
  Keyword arguments: {}


NameError: name 'absorbance' is not defined

# Save results to dataframe

In [ ]:
hyperspectrum =pd.DataFrame(absorbance_list)
hyperspectrum.columns = wavenumber
hyperspectrum['map_x'] = df['map_x'].values
hyperspectrum['map_y'] = df['map_y'].values

In [ ]:
reconstructed =pd.DataFrame(reconstructed_list)
reconstructed.columns = wavenumber
reconstructed['map_x'] = df['map_x'].values
reconstructed['map_y'] = df['map_y'].values

In [ ]:
bvae_df =pd.DataFrame()
bvae_df['map_x'] = df['map_x'].values
bvae_df['map_y'] = df['map_y'].values
bvae_df['L1'] = L1_list
bvae_df['L2'] = L2_list
bvae_df['L3'] = L3_list
bvae_df['MSE'] = MSE_list

# Save to csv files for viewing in Quasar

In [ ]:
path_to_save = 'C:/Users/Zach/Documents/BVAE_processed/'

bvae_df.to_csv(path_to_save +'bvae_'+filename)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Hyperspectra/"
file=os.path.join(path_to_save + 'hyperspectrum_'+filename)
hyperspectrum.to_csv(file, index=False)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Reconstructions/"
reconstructed.to_csv(path_to_save +'reconstructed_'+filename, index=False)